# Fine-tuning Stable Diffusion XL with Dreambooth and LoRA

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/17LHoxqYGjwX-K0LgJkhbGeZ4tHqJJ3Gr?usp=sharing)
[![Open in SageMaker Studio](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/adhiiisetiawan/SYN_DAM-TUTORIAL-PRICAI-2024/blob/main/Diffusion_Generate_Data.ipynb)

This notebook demonstrates how to fine-tune Stable Diffusion models using the Dreambooth technique. It covers two specific models:

1. **Stable Diffusion XL (SDXL)**: This section shows the process of fine-tuning SDXL using a LoRA (Low-Rank Adaptation) approach. It utilizes a custom dataset and focuses on training with a specific instance prompt.

2. **Stable Diffusion 3 (SD3)**: This section guides through fine-tuning SD3 using LoRA as well.  It leverages a pre-existing dog image dataset and demonstrates training with a personalized prompt.

The notebook uses the `diffusers` library from Hugging Face, along with the `accelerate` library for efficient training.  It also integrates with `wandb` for experiment tracking.

**Key goals of this notebook:**

* Understand the steps involved in fine-tuning Stable Diffusion models for image generation.
* Learn how to apply Dreambooth LoRA training for both SDXL and SD3.
* Leverage existing datasets and customize training parameters.
* Track training progress using `wandb`.

Let's explore the code and learn how to personalize these powerful image generation models!

In [ ]:
!nvidia-smi

Sat Oct 12 09:04:55 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        On  |   00000000:02:00.0 Off |                  Off |
|  0%   27C    P8             14W /  450W |       2MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install huggingface_hub wandb datasets bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 5.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 30.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.4.0
    Uninstalling fsspec-2023.4.0:
      Successfully uninstalled fsspec-2023.4.0

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
!pip install --upgrade Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 19.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.3.0
    Uninstalling Pillow-9.3.0:
      Successfully uninstalled Pillow-9.3.0

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
!git clone https://github.com/huggingface/diffusers

Cloning into 'diffusers'...
remote: Enumerating objects: 71873, done.
remote: Counting objects: 100% (10654/10654), done.
remote: Compressing objects: 100% (985/985), done.
remote: Total 71873 (delta 10210), reused 9776 (delta 9603), pack-reused 61219 (from 1)
Receiving objects: 100% (71873/71873), 49.99 MiB | 21.34 MiB/s, done.
Resolving deltas: 100% (53306/53306), done.
Updating files: 100% (1584/1584), done.


In [ ]:
%cd diffusers

/workspace/diffusers


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
!ls

CITATION.cff	    MANIFEST.in    _typos.toml	examples	src
CODE_OF_CONDUCT.md  Makefile	   benchmarks	pyproject.toml	tests
CONTRIBUTING.md     PHILOSOPHY.md  docker	scripts		utils
LICENSE		    README.md	   docs		setup.py


In [ ]:
!pip install -e .

Obtaining file:///workspace/diffusers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.7/782.7 kB 6.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 55.4 MB/s eta 0:00:00
  Building editable for diffusers (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.31.0.dev0-0.editable-py3-none-any.whl size=11102 sha256=8b586ba1d320161fadf6da1cdc8ef0fa9eb4b376762d6e18d14d1859e947e4a2
  Stored in directory: /tmp/pip-ephem-wheel-cache-19u57b4e/wheels/86/d0/a3/783dfd30a3a65321046424bb8e359b826b7cf724c77d331554
Successfully built diffusers

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade

In [ ]:
%cd examples/dreambooth

/workspace/diffusers/examples/dreambooth


In [ ]:
!pip install -r requirements_sd3.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 4.6 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 61.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 128.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 260.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 136.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 27.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 332.1 MB/s eta 0:00:00

In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()

PosixPath('/root/.cache/huggingface/accelerate/default_config.yaml')

# SDXL

In [ ]:
!ls

README.md		 test_dreambooth_lora_edm.py
README_flux.md		 test_dreambooth_lora_flux.py
README_sd3.md		 test_dreambooth_lora_sd3.py
README_sdxl.md		 test_dreambooth_sd3.py
requirements.txt	 train_dreambooth.py
requirements_flax.txt	 train_dreambooth_flax.py
requirements_flux.txt	 train_dreambooth_flux.py
requirements_sd3.txt	 train_dreambooth_lora.py
requirements_sdxl.txt	 train_dreambooth_lora_flux.py
test_dreambooth.py	 train_dreambooth_lora_sd3.py
test_dreambooth_flux.py  train_dreambooth_lora_sdxl.py
test_dreambooth_lora.py  train_dreambooth_sd3.py


In [ ]:
!pip3 install -r requirements_sdxl.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 1.1 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: peft
    Found existing installation: peft 0.11.1
    Uninstalling peft-0.11.1:
      Successfully uninstalled peft-0.11.1

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


Here's a table with all the arguments for the `train_dreambooth_lora_sdxl.py` script:

| Argument                             | Description                                     | Example Value                                    |
|--------------------------------------|-------------------------------------------------|--------------------------------------------------|
| `--pretrained_model_name_or_path`    | Path or identifier for the pretrained model     | `"stabilityai/stable-diffusion-xl-base-1.0"`     |
| `--dataset_name`                     | Name of the dataset to use                      | `"adhisetiawan/food-bakso-img"`                  |
| `--pretrained_vae_model_name_or_path`| Path or identifier for the pretrained VAE model | `"madebyollin/sdxl-vae-fp16-fix"`                |
| `--output_dir`                       | Directory to save the trained model             | `"lora-trained-xl"`                              |
| `--use_8bit_adam`                    | Use 8-bit Adam optimizer                        | Flag only (no value)                             |
| `--mixed_precision`                  | Type of mixed precision                         | `"bf16"`                                         |
| `--train_text_encoder`               | Train the text encoder                          | Flag only (no value)                             |
| `--instance_prompt`                  | Instance prompt for training                    | `"a photo of bakso"`                             |
| `--resolution`                       | Resolution for training images                  | `1024`                                           |
| `--train_batch_size`                 | Batch size for training                         | `1`                                              |
| `--gradient_accumulation_steps`      | Steps for gradient accumulation                 | `4`                                              |
| `--learning_rate`                    | Learning rate for training                      | `1e-4`                                           |
| `--lr_scheduler`                     | Learning rate scheduler                         | `"constant"`                                     |
| `--lr_warmup_steps`                  | Warmup steps for learning rate                  | `0`                                              |
| `--max_train_steps`                  | Maximum number of training steps                | `500`                                            |
| `--validation_prompt`                | Prompt used for validation                      | `"A photo of bakso in a bowl"`                   |
| `--validation_epochs`                | Epoch interval for validation                   | `5`                                              |
| `--report_to`                        | Reporting platform (e.g., `wandb`)              | `"wandb"`                                        |
| `--seed`                             | Random seed for reproducibility                 | `"0"`                                            |
| `--hub_model_id`                     | Model identifier for pushing to the hub         | `"sdxl-base-test"`                               |
| `--push_to_hub`                      | Push the model to the hub                       | Flag only (no value)                             |

This table summarizes each argument, its description, and example values.

In [ ]:
!accelerate launch train_dreambooth_lora_sdxl.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0"  \
  --dataset_name="adhisetiawan/food-bakso-img" \
  --pretrained_vae_model_name_or_path="madebyollin/sdxl-vae-fp16-fix" \
  --output_dir="lora-trained-xl" \
  --use_8bit_adam \
  --mixed_precision="bf16" \
  --train_text_encoder \
  --instance_prompt="a photo of bakso" \
  --resolution=1024 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=500 \
  --validation_prompt="A photo of bakso in a bowl" \
  --validation_epochs=5 \
  --report_to="wandb" \
  --seed="0" \
  --hub_model_id="sdxl-base-test" \
  --push_to_hub

# Stable Diffusion 3

In [ ]:
!export MODEL_NAME="stabilityai/stable-diffusion-3-medium-diffusers"
!export INSTANCE_DIR="dog/"
!export OUTPUT_DIR="trained-sd3-lora"

In [ ]:
!ls

README_flux.md	       requirements.txt		     train_dreambooth_flax.py
README.md	       test_dreambooth_flux.py	     train_dreambooth_flux.py
README_sd3.md	       test_dreambooth_lora_edm.py   train_dreambooth_lora_flux.py
README_sdxl.md	       test_dreambooth_lora_flux.py  train_dreambooth_lora.py
requirements_flax.txt  test_dreambooth_lora.py	     train_dreambooth_lora_sd3.py
requirements_flux.txt  test_dreambooth_lora_sd3.py   train_dreambooth_lora_sdxl.py
requirements_sd3.txt   test_dreambooth.py	     train_dreambooth.py
requirements_sdxl.txt  test_dreambooth_sd3.py	     train_dreambooth_sd3.py


In [ ]:
import wandb
wandb.require("core")

In [ ]:
!accelerate launch train_dreambooth_lora_sd3.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-3-medium-diffusers"  \
  --dataset_name="diffusers/dog-example" \
  --output_dir="trained-sd3-lora" \
  --mixed_precision="fp16" \
  --instance_prompt="a photo of sks dog" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-5 \
  --report_to="wandb" \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=500 \
  --validation_prompt="A photo of sks dog in a bucket" \
  --validation_epochs=25 \
  --seed="0" \
  --push_to_hub

2024-08-15 11:33:11.715459: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-15 11:33:11.732157: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-15 11:33:11.753417: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-15 11:33:11.759931: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-15 11:33:11.775496: I tensorflow/core/platform/cpu_feature_guar